This notebook takes the synthetic data and partitions it into
the "question" and IDV subsets for the R vignette.

In [14]:
import pandas as pd
import numpy as nd
import os
import glob 

cwd = os.getcwd()

print(cwd)
print(glob.glob(cwd + "/data/*" ))

/Users/Promachos/Dropbox (Personal)/IRTM-Pkg/IRTM-Synth/IRTM-Synth
['/Users/Promachos/Dropbox (Personal)/IRTM-Pkg/IRTM-Synth/IRTM-Synth/data/synth_questions.csv', '/Users/Promachos/Dropbox (Personal)/IRTM-Pkg/IRTM-Synth/IRTM-Synth/data/mcodes.rda', '/Users/Promachos/Dropbox (Personal)/IRTM-Pkg/IRTM-Synth/IRTM-Synth/data/EB_M.rds', '/Users/Promachos/Dropbox (Personal)/IRTM-Pkg/IRTM-Synth/IRTM-Synth/data/eurob_toy.csv', '/Users/Promachos/Dropbox (Personal)/IRTM-Pkg/IRTM-Synth/IRTM-Synth/data/EB_Y.rds', '/Users/Promachos/Dropbox (Personal)/IRTM-Pkg/IRTM-Synth/IRTM-Synth/data/ebdatDemographVaribs.rda', '/Users/Promachos/Dropbox (Personal)/IRTM-Pkg/IRTM-Synth/IRTM-Synth/data/eurob_sample_ids.csv', '/Users/Promachos/Dropbox (Personal)/IRTM-Pkg/IRTM-Synth/IRTM-Synth/data/SI395.csv', '/Users/Promachos/Dropbox (Personal)/IRTM-Pkg/IRTM-Synth/IRTM-Synth/data/synth_idvs.csv', '/Users/Promachos/Dropbox (Personal)/IRTM-Pkg/IRTM-Synth/IRTM-Synth/data/synth_test.csv']


In [15]:
#datapath = "/data/"
ebsynth = pd.read_csv("./data/synth_test.csv")

print(ebsynth.shape) ## 3000 X 172

(3000, 172)


Subset the IDVs, and undo the numeric recoding of the string variables that we had to do for YSynth.

In [16]:
## Code from previous script:

## Will need 'isocntry'
trust = [col for col in ebsynth.columns if col.startswith('qa6a')] ## trust in media
covidinfo = [col for col in ebsynth if col.startswith('qa20')] ## who trust for covid news
news= [col for col in ebsynth if col.startswith('qd4a')] ## news consumption
social_class = [col for col in ebsynth if col.startswith('d63')]# self-identified social class 
pol_oriet = [col for col in ebsynth if col.startswith('d1')] ## these are demographics (education, employment, gender)
    
idvs_subset = (['uniqid', 'd63', 'class', 'polorient',
                'mediatrust', 'trustnom', 'trustallm',
                'trustwebonly', 'trusttradm'] + trust + covidinfo + news + social_class + pol_oriet)

print(idvs_subset) ## Note that this is a set, so no repetitions

idvs = ebsynth[idvs_subset] ## 

print(idvs.shape) ## 3000 x 26

['uniqid', 'd63', 'class', 'polorient', 'mediatrust', 'trustnom', 'trustallm', 'trustwebonly', 'trusttradm', 'qa6a_1', 'qa6a_2', 'qa6a_5', 'qa6a_3', 'qa6at', 'qa6a_4', 'd63', 'd15a', 'd1r2', 'd1', 'd11r1', 'd10', 'd1r1', 'd15a_r1', 'd11', 'd11r2', 'd15a_r2']
(3000, 26)


Re-map the string values to the IDVs (from numeric format needed for the synth). 
This is for convenience:

In [17]:
## NOTE -- 5/6/24 I caught a coding error in the original; which I'll have to go through and update
#idvs["class"].value_counts() 
 
idvs.loc[idvs['class'] == 1, 'class'] = 'WorkingClass'
idvs.loc[idvs['class'] == 2, 'class'] = 'LowerMiddle'
idvs.loc[idvs['class'] == 3, 'class'] = 'MiddleClass'
idvs.loc[idvs['class'] == 4, 'class'] = 'UpperMiddleClass'
idvs.loc[idvs['class'] == 5, 'class'] = 'UpperClass'
#idvs.loc[idvs['class'].isin([6, 7, 8, 9]), 'class'] = 5 #'Other/None/DN/Refused' (This should have been coded as 6)

idvs["class"].value_counts() 

class
MiddleClass         1161
WorkingClass         828
LowerMiddle          483
UpperMiddleClass     395
UpperClass           133
Name: count, dtype: int64

In [18]:
## Political orientation
idvs.loc[idvs['polorient'] == 1, 'polorient'] = 'FarLeft'
idvs.loc[idvs['polorient'] == 2, 'polorient'] = 'Left'
idvs.loc[idvs['polorient'] == 3, 'polorient'] = 'Center'
idvs.loc[idvs['polorient'] == 4, 'polorient'] = 'Right'
idvs.loc[idvs['polorient'] == 5, 'polorient'] = 'FarRight'
idvs.loc[idvs['polorient'] == 6, 'polorient'] = 'Refused/DNt'

idvs["polorient"].value_counts() 
#idvs["polorient"].describe()

polorient
Left           1211
Center          869
Right           339
Refused/DNt     242
FarRight        194
FarLeft         145
Name: count, dtype: int64

In [19]:
idvs.loc[idvs['mediatrust'] == 1, 'mediatrust'] = 'TrustTrad' #
idvs.loc[idvs['mediatrust'] == 2, 'mediatrust'] = 'TrustWebOnly' #
idvs.loc[idvs['mediatrust'] == 3, 'mediatrust'] = 'TrustAll' #
idvs.loc[idvs['mediatrust'] == 4, 'mediatrust'] = 'TrustNone' #
idvs.loc[idvs['mediatrust'] == 9, 'mediatrust'] = 'Other' #
print(idvs['mediatrust'].value_counts())

mediatrust
TrustTrad       1192
TrustNone        866
TrustAll         611
TrustWebOnly     169
Other            162
Name: count, dtype: int64


In [20]:
print(idvs['trustnom'].value_counts()) ## 0/1

trustnom
0    2582
1     418
Name: count, dtype: int64


In [21]:
idvs.describe


<bound method NDFrame.describe of          uniqid  d63             class polorient mediatrust  trustnom  \
0     320387826    3       MiddleClass  FarRight      Other         0   
1     394380111    4      WorkingClass    Center   TrustAll         0   
2     139929284    1       MiddleClass    Center  TrustTrad         0   
3     424648035    3       LowerMiddle      Left   TrustAll         0   
4     478547108    3      WorkingClass    Center  TrustTrad         0   
...         ...  ...               ...       ...        ...       ...   
2995  391427319    3        UpperClass      Left  TrustTrad         0   
2996  100729289    3       MiddleClass  FarRight  TrustNone         0   
2997  431738794    3      WorkingClass      Left  TrustTrad         0   
2998  309079928    3  UpperMiddleClass    Center  TrustTrad         0   
2999  252505987    1       LowerMiddle      Left  TrustNone         0   

      trustallm  trustwebonly  trusttradm  qa6a_1  ...  d15a  d1r2  d1  d11r1  \
0       

In [22]:
idvs.to_csv('./data/synth_idvs.csv')

In [23]:
ebsynth = ebsynth.drop(idvs_subset, axis = 1)

In [11]:
print(ebsynth.columns)

Index(['Unnamed: 0', 'd73_4', 'qa10_3', 'qd5_14', 'qa4a_5', 'qa2a_4',
       'polintr_4', 'qa2a_5', 'd40b', 'qa6b_3',
       ...
       'qd5_3', 'd62t', 'qa18_2', 'd73_3', 'qa4a_7', 'sd18a', 'qa4a_10',
       'qa7_2', 'qa1a_7', 'qa4a_2'],
      dtype='object', length=147)


66


In [24]:
ebsynth.to_csv('./data/synth_questions.csv')